## 准备数据

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    return (x, y), (x_test, y_test)

In [7]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [8]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal(shape=(28 * 28, 512), stddev=0.05), trainable=True)
        self.b1 = tf.Variable(tf.zeros(shape=(512,)), trainable=True)
        self.W2 = tf.Variable(tf.random.normal(shape=(512, 10), stddev=0.05), trainable=True)
        self.b2 = tf.Variable(tf.zeros(shape=(10,)), trainable=True)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x_flattened = tf.reshape(x, [-1, 784])
        h1 = tf.matmul(x_flattened, self.W1) + self.b1
        activated_h1 = tf.tanh(h1)
        logits = tf.matmul(activated_h1, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [9]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [10]:
train_data, test_data = mnist_dataset()
for epoch in range(400):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.393965 ; accuracy 0.061983332
epoch 1 : loss 2.3701549 ; accuracy 0.0687
epoch 2 : loss 2.347191 ; accuracy 0.076516666
epoch 3 : loss 2.324986 ; accuracy 0.08528333
epoch 4 : loss 2.3034635 ; accuracy 0.09463333
epoch 5 : loss 2.2825584 ; accuracy 0.10408334
epoch 6 : loss 2.262213 ; accuracy 0.11586667
epoch 7 : loss 2.242379 ; accuracy 0.12791666
epoch 8 : loss 2.2230124 ; accuracy 0.14101666
epoch 9 : loss 2.2040763 ; accuracy 0.15543333
epoch 10 : loss 2.185538 ; accuracy 0.17115
epoch 11 : loss 2.1673691 ; accuracy 0.18895
epoch 12 : loss 2.1495447 ; accuracy 0.2077
epoch 13 : loss 2.1320431 ; accuracy 0.22695
epoch 14 : loss 2.1148455 ; accuracy 0.2476
epoch 15 : loss 2.097935 ; accuracy 0.26905
epoch 16 : loss 2.0812972 ; accuracy 0.2908
epoch 17 : loss 2.0649195 ; accuracy 0.31238332
epoch 18 : loss 2.0487907 ; accuracy 0.33338332
epoch 19 : loss 2.0329006 ; accuracy 0.3532
epoch 20 : loss 2.0172405 ; accuracy 0.37326667
epoch 21 : loss 2.0018032 ; accuracy 0.